In [1]:
import pandas as pd
from sqlalchemy.exc import SQLAlchemyError # Para capturar errores específicos de SQLAlchemy si es necesario
from src.conexion_bd import ConexionBD  # Ajusta la ruta si es necesario

print("--- Demostración del Sistema de Análisis de Ventas ---")

--- Demostración del Sistema de Análisis de Ventas ---


In [2]:
print("\n--- 1. Probando Conexión a la Base de Datos (Singleton) ---")
try:
    conector_bd1 = ConexionBD()
    motor1 = conector_bd1.obtener_motor()

    conector_bd2 = ConexionBD()
    motor2 = conector_bd2.obtener_motor()

    if motor1 and motor2:
        print(f"Instancia 1 del conector: {id(conector_bd1)}")
        print(f"Instancia 2 del conector: {id(conector_bd2)}")
        print(f"¿Son la misma instancia? {'Sí' if conector_bd1 is conector_bd2 else 'No'}")
        print(f"Motor de la instancia 1: {motor1}")
        print(f"Motor de la instancia 2: {motor2}")
        print(f"¿Son el mismo motor? {'Sí' if motor1 is motor2 else 'No'}")
        
        # Probar una conexión simple
        with motor1.connect() as connection:
            print("Conexión directa al motor exitosa.")
            
    else:
        print("Error: No se pudo obtener el motor de la base de datos. Revisa las credenciales y la configuración.")

except ValueError as ve:
    print(f"Error de configuración: {ve}")
except Exception as e:
    print(f"Error inesperado al conectar: {e}")


--- 1. Probando Conexión a la Base de Datos (Singleton) ---
INFO     | sistema_ventas | Sistema de logging configurado correctamente
INFO     | sistema_ventas | Nivel de logging: INFO
INFO     | sistema_ventas | Directorio de logs: d:\gnavarro\Escritorio\sistema-de-analisis-de-ventas\logs
INFO     | sistema_ventas.GestorCache | Cache de consultas inicializado - Habilitado: True
INFO     | sistema_ventas.GestorCache | Configuración: duración=15min, tamaño_max=100
INFO     | sistema_ventas.ConexionBD | INICIANDO: inicialización de conexión a base de datos
INFO     | sistema_ventas.ConexionBD | Conexión a la base de datos establecida exitosamente.
INFO     | sistema_ventas.ConexionBD | Pool configurado: size=5, overflow=10, query_timeout_config=30s
INFO     | sistema_ventas.Estadisticas | MÉTRICAS BD - Conexiones activas: 1, Consultas ejecutadas: 0
INFO     | sistema_ventas.ConexionBD | Cache inicializado - Habilitado: True
INFO     | sistema_ventas.ConexionBD | COMPLETADO: inicializació

In [3]:
print("\n--- 2. Resultados de Consultas SQL ---")

# (conector_bd1 fue instanciado en una celda anterior del notebook)
if 'conector_bd1' in locals() and conector_bd1.obtener_motor(): # Verifica si el motor está disponible
    print("\nConsulta: Primeros 5 clientes")

    df_clientes = conector_bd1.ejecutar_consulta("SELECT * FROM Customers LIMIT 5;") 
    if df_clientes is not None:
        display(df_clientes)
    else:
        print("No se pudieron obtener los clientes.")

    print("\nConsulta: Productos con precio (Price) mayor a 50")
    # Usamos "Products" para la tabla y "ProductName", "Price" para las columnas.
    df_productos_caros = conector_bd1.ejecutar_consulta(
        "SELECT ProductName, Price FROM Products WHERE Price > 50 ORDER BY Price DESC LIMIT 5;" 
    )
    if df_productos_caros is not None:
        display(df_productos_caros)
    else:
        print("No se pudieron obtener los productos caros.")
else:
    print("No se pueden ejecutar consultas, la conexión a la BD no está establecida o `conector_bd1` no está definido.")


--- 2. Resultados de Consultas SQL ---

Consulta: Primeros 5 clientes
INFO     | sistema_ventas.GestorCacheConsultas | INICIANDO: búsqueda en cache de consulta
INFO     | sistema_ventas.GestorCacheConsultas | COMPLETADO: búsqueda en cache de consulta (duración: 0.001s)
INFO     | sistema_ventas.GestorCacheConsultas | INICIANDO: almacenamiento en cache de consulta
INFO     | sistema_ventas.GestorCacheConsultas | COMPLETADO: almacenamiento en cache de consulta (duración: 0.003s)
INFO     | sistema_ventas.ConexionBD | Consulta exitosa: 5 filas en 0.108s


,CustomerID,FirstName,MiddleInitial,LastName,Address,CityID
0,1,Stefanie,Y,Frye,97 Oak Avenue,79
1,2,Sandy,T,Kirby,52 White First Freeway,96
2,3,Lee,T,Zhang,921 White Fabien Avenue,55
3,4,Regina,S,Avery,75 Old Avenue,40
4,5,Daniel,S,Mccann,283 South Green Hague Avenue,2



Consulta: Productos con precio (Price) mayor a 50
INFO     | sistema_ventas.GestorCacheConsultas | INICIANDO: búsqueda en cache de consulta
INFO     | sistema_ventas.GestorCacheConsultas | COMPLETADO: búsqueda en cache de consulta (duración: 0.001s)
INFO     | sistema_ventas.GestorCacheConsultas | INICIANDO: almacenamiento en cache de consulta
INFO     | sistema_ventas.GestorCacheConsultas | COMPLETADO: almacenamiento en cache de consulta (duración: 0.004s)
INFO     | sistema_ventas.ConexionBD | Consulta exitosa: 5 filas en 0.038s


,ProductName,Price
0,Shrimp - 31/40,99.8755
1,Beef - Inside Round,99.3193
2,Puree - Passion Fruit,98.8263
3,Bread - Calabrese Baguette,98.5978
4,Zucchini - Yellow,98.4644


In [4]:
#Análisis Avanzado - Ranking de Productos por Ventas Dentro de Cada Categoría (Cálculo de ventas DESDE Products.Price CON DESCUENTO)
print("\n--- 3.1. Análisis Avanzado: Top Productos por Categoría (Cálculo desde Products.Price con Descuento) ---")

if 'conector_bd1' in locals() and conector_bd1.obtener_motor():
    # Asumimos que Sales.Discount es un decimal (ej. 0.10 para 10%)
    # Usamos IFNULL(s.Discount, 0) para tratar los NULL como sin descuento (0%)
    # Usamos p.Price de la tabla Products como precio base.
    consulta_avanzada_sql = """
    WITH VentasProducto AS (
        -- Paso 1: Calcular el monto total de ventas netas para cada producto
        SELECT
            p.ProductID,
            p.ProductName,
            p.CategoryID,
            SUM(s.Quantity * p.Price * (1 - IFNULL(s.Discount, 0))) AS monto_total_ventas_producto
        FROM Sales s
        JOIN Products p ON s.ProductID = p.ProductID
        GROUP BY
            p.ProductID,
            p.ProductName,
            p.CategoryID
    ),
    VentasProductoCategoriaConNombre AS (
        -- Paso 2: Unir con la tabla de categorías para obtener el nombre de la categoría
        SELECT
            vp.ProductID,
            vp.ProductName,
            vp.CategoryID,
            cat.CategoryName,
            vp.monto_total_ventas_producto
        FROM VentasProducto vp
        JOIN Categories cat ON vp.CategoryID = cat.CategoryID
    ),
    RankingVentas AS (
        -- Paso 3: Rankear los productos dentro de cada categoría según su monto total de ventas netas
        SELECT
            vpc.ProductID,
            vpc.ProductName,
            vpc.CategoryName,
            vpc.monto_total_ventas_producto,
            RANK() OVER (PARTITION BY vpc.CategoryName ORDER BY vpc.monto_total_ventas_producto DESC) AS ranking_en_categoria
        FROM VentasProductoCategoriaConNombre vpc
    )
    -- Paso 4: Seleccionar el resultado final, mostrando por ejemplo el Top 3 de productos por categoría
    SELECT
        CategoryName AS "Categoría",
        ProductName AS "Producto",
        monto_total_ventas_producto AS "Monto Total de Ventas Netas",
        ranking_en_categoria AS "Ranking en Categoría"
    FROM RankingVentas
    WHERE ranking_en_categoria <= 3  -- Puedes ajustar este número
    ORDER BY
        "Categoría",
        ranking_en_categoria;
    """
    
    print("\nConsulta Avanzada: Top 3 Productos por Ventas Netas en Cada Categoría (Usando Products.Price y Sales.Discount)")
    # print("SQL a ejecutar:\n", consulta_avanzada_sql) # Descomenta esto para ver el SQL exacto que se envía
    df_ranking_productos = conector_bd1.ejecutar_consulta(consulta_avanzada_sql)
    
    if df_ranking_productos is not None and not df_ranking_productos.empty:
        print("¡Consulta ejecutada exitosamente!")
        display(df_ranking_productos)
    elif df_ranking_productos is not None and df_ranking_productos.empty:
        print("La consulta se ejecutó correctamente pero no arrojó resultados.")
    else:
        print("No se pudo obtener el ranking de productos. Revisa el mensaje de error anterior de la base de datos.")
else:
    print("No se puede ejecutar la consulta avanzada, la conexión a la BD no está establecida o `conector_bd1` no está definido.")


--- 3.1. Análisis Avanzado: Top Productos por Categoría (Cálculo desde Products.Price con Descuento) ---

Consulta Avanzada: Top 3 Productos por Ventas Netas en Cada Categoría (Usando Products.Price y Sales.Discount)
INFO     | sistema_ventas.ConexionBD | Consulta exitosa: 33 filas en 0.116s
¡Consulta ejecutada exitosamente!


,Categoría,Producto,Monto Total de Ventas Netas,Ranking en Categoría
0,Beverages,Placemat - Scallop; White,191.57300,1
1,Produce,Orange - Canned; Mandarin,157.68100,1
2,Snails,Cookie Dough - Double,222.19829,1
3,Cereals,Shrimp - 31/40,199.75100,1
4,Grain,Bread - Multigrain,193.04000,1
5,Shell fish,Lettuce - Spring Mix,90.01690,1
6,Confections,Juice - Lime,183.56660,1
7,Meat,Cod - Black Whole Fillet,290.45320,1
8,Seafood,Cheese - Wine,148.81520,1
9,Dairy,Scampi Tail,380.38280,1


In [5]:
# --- 4. Demostración del Patrón Factory (FabricaModelos) ---
print("\\n--- 4. Demostración del Patrón Factory (FabricaModelos) ---")

from src.utils.fabrica_modelo import FabricaModelos #
from src.modelos.cliente import Cliente #

# Crear una instancia de la fábrica
fabrica = FabricaModelos()

# Datos de ejemplo para crear un cliente
datos_cliente_ejemplo = {
    'CustomerID': 999,
    'FirstName': 'Gustavo',
    'MiddleInitial': 'A',
    'LastName': 'Navarro',
    'CityID': 1, # Asume que CityID 1 existe
    'Address': 'Calle Demo 123'
}

print(f"\\nCreando un objeto Cliente usando FabricaModelos con datos: {datos_cliente_ejemplo}")

try:
    cliente_creado = fabrica.create_from_dict('cliente', datos_cliente_ejemplo) #
    print("\\nCliente creado exitosamente:")
    print(f"  ID: {cliente_creado.id_cliente}") #
    print(f"  Nombre Completo: {cliente_creado.nombre_completo()}") #
    print(f"  Dirección: {cliente_creado.direccion}") #
    print(f"  Tipo de objeto: {type(cliente_creado)}")
except ValueError as ve:
    print(f"Error al crear cliente con la fábrica: {ve}")
except Exception as e:
    print(f"Un error inesperado ocurrió: {e}")

# Ejemplo con un DataFrame 
import pandas as pd
datos_df_ejemplo = pd.DataFrame([
    {
        'CustomerID': 1000, 'FirstName': 'Ana', 'LastName': 'López', 'CityID': 1,
        'MiddleInitial': 'B', 'Address': 'Calle Mayor 123'
    }
])
print(f"\\nCreando objetos Cliente desde un DataFrame usando FabricaModelos:")
try:
    clientes_desde_df = fabrica.create_multiple_from_dataframe('cliente', datos_df_ejemplo) #
    if clientes_desde_df:
        print("Clientes creados desde DataFrame:")
        for cliente_df in clientes_desde_df:
            print(f"  - {cliente_df.nombre_completo()}, ID: {cliente_df.id_cliente}")
    else:
        print("No se crearon clientes desde el DataFrame.")
except ValueError as ve:
    print(f"Error al crear clientes desde DataFrame: {ve}")
except Exception as e:
    print(f"Un error inesperado ocurrió: {e}")

\n--- 4. Demostración del Patrón Factory (FabricaModelos) ---
INFO     | sistema_ventas.FabricaModelos | FabricaModelos inicializada correctamente
\nCreando un objeto Cliente usando FabricaModelos con datos: {'CustomerID': 999, 'FirstName': 'Gustavo', 'MiddleInitial': 'A', 'LastName': 'Navarro', 'CityID': 1, 'Address': 'Calle Demo 123'}
INFO     | sistema_ventas.FabricaModelos | INICIANDO: creación de modelo desde diccionario
INFO     | sistema_ventas.FabricaModelos | Modelo cliente creado exitosamente (ID: 999)
INFO     | sistema_ventas.FabricaModelos | COMPLETADO: creación de modelo desde diccionario (duración: 0.006s)
\nCliente creado exitosamente:
  ID: 999
  Nombre Completo: Gustavo A. Navarro
  Dirección: Calle Demo 123
  Tipo de objeto: <class 'src.modelos.cliente.Cliente'>
\nCreando objetos Cliente desde un DataFrame usando FabricaModelos:
INFO     | sistema_ventas.FabricaModelos | INICIANDO: creación múltiple de modelos desde DataFrame
INFO     | sistema_ventas.FabricaModelos 

In [6]:
# --- 5. Demostración del Patrón Builder (ConstructorConsultaSQL) ---
print("\\n--- 5. Demostración del Patrón Builder (ConstructorConsultaSQL) ---")

from src.utils.constructor_consulta import ConstructorConsultaSQL #

# Crear una instancia del constructor
constructor_sql = ConstructorConsultaSQL()

# Construir una consulta paso a paso
print("\\nConstruyendo una consulta SQL para 'Empleados de la ciudad con ID 7':")
consulta_construida = (constructor_sql
                       .seleccionar("EmployeeID", "FirstName", "LastName", "HireDate") #
                       .desde_tabla("employees") #
                       .donde("CityID = 7") #
                       .ordenar_por("HireDate", "DESC") #
                       .limite(5) #
                       .construir()) #

print("\\nConsulta SQL construida:")
print(consulta_construida)

# Opcionalmente, ejecutar la consulta construida (asumiendo que conector_bd1 ya está definido y conectado)
print("\\nEjecutando la consulta construida (si la conexión está activa):")
if 'conector_bd1' in locals() and conector_bd1.obtener_motor():
    try:
        df_resultado_builder = conector_bd1.ejecutar_consulta(consulta_construida)
        if df_resultado_builder is not None:
            display(df_resultado_builder)
        else:
            print("La consulta construida no devolvió resultados o hubo un error.")
    except Exception as e:
        print(f"Error al ejecutar la consulta construida: {e}")
else:
    print("No se puede ejecutar la consulta, la conexión a la BD no está establecida o `conector_bd1` no está definido.")
    
# LÍNEA DE DEPURACIÓN A AGREGAR:
print(f"DEBUG: Tipo de constructor_sql ANTES de llamar a .reiniciar(): {type(constructor_sql)}")

# Demostrar reinicio y construcción de otra consulta
print("\\nDemostrando reutilización del constructor después de reiniciar:")
consulta_productos = (constructor_sql.reiniciar() #
                      .seleccionar("ProductName", "Price")
                      .desde_tabla("products")
                      .donde("Price < 10")
                      .ordenar_por("Price", "ASC")
                      .limite(3)
                      .construir())
print("\\nNueva consulta SQL construida para productos:")
print(consulta_productos)

\n--- 5. Demostración del Patrón Builder (ConstructorConsultaSQL) ---
\nConstruyendo una consulta SQL para 'Empleados de la ciudad con ID 7':
\nConsulta SQL construida:
SELECT EmployeeID, FirstName, LastName, HireDate
FROM employees
WHERE CityID = 7
ORDER BY HireDate DESC
LIMIT 5;
\nEjecutando la consulta construida (si la conexión está activa):
INFO     | sistema_ventas.GestorCacheConsultas | INICIANDO: búsqueda en cache de consulta
INFO     | sistema_ventas.GestorCacheConsultas | COMPLETADO: búsqueda en cache de consulta (duración: 0.001s)
INFO     | sistema_ventas.GestorCacheConsultas | INICIANDO: almacenamiento en cache de consulta
INFO     | sistema_ventas.GestorCacheConsultas | COMPLETADO: almacenamiento en cache de consulta (duración: 0.005s)
INFO     | sistema_ventas.ConexionBD | Consulta exitosa: 1 filas en 0.064s


,EmployeeID,FirstName,LastName,HireDate
0,23,Janet,Flowers,2010-05-12


DEBUG: Tipo de constructor_sql ANTES de llamar a .reiniciar(): <class 'src.utils.constructor_consulta.ConstructorConsultaSQL'>
\nDemostrando reutilización del constructor después de reiniciar:
\nNueva consulta SQL construida para productos:
SELECT ProductName, Price
FROM products
WHERE Price < 10
ORDER BY Price ASC
LIMIT 3;


In [7]:
# --- 6. Ejecución de Pruebas Unitarias (pytest) ---
print("\\n--- 6. Ejecución de Pruebas Unitarias (pytest) ---")
print("Ejecutando pytest para todas las pruebas en el directorio 'tests/'...")
print("La salida de las pruebas se mostrará a continuación:")

# Usar ! para ejecutar comandos de shell.
# El modificador -v es para salida verbosa, --tb=short para un traceback más corto.
!pytest tests/ -v --tb=short

\n--- 6. Ejecución de Pruebas Unitarias (pytest) ---
Ejecutando pytest para todas las pruebas en el directorio 'tests/'...
La salida de las pruebas se mostrará a continuación:
============================= test session starts =============================
platform win32 -- Python 3.10.11, pytest-7.4.3, pluggy-1.6.0 -- D:\gnavarro\Escritorio\sistema-de-analisis-de-ventas\venv\Scripts\python.exe
cachedir: .pytest_cache
rootdir: d:\gnavarro\Escritorio\sistema-de-analisis-de-ventas
configfile: pytest.ini
plugins: anyio-4.9.0, cov-4.1.0, mock-3.12.0
collecting ... collected 22 items

tests/test_conexion_bd.py::test_singleton_conexion_bd PASSED             [  4%]
tests/test_conexion_bd.py::test_ejecutar_consulta_sin_conexion PASSED    [  9%]
tests/test_empleado.py::test_empleado_describir_antiguedad_varios_casos PASSED [ 13%]
tests/test_integridad_datos.py::test_conteo_paises PASSED                [ 18%]
tests/test_integridad_datos.py::test_conteo_ventas PASSED                [ 22%]
tests/te

In [ ]:
# Consulta Avanzada 1: Ranking de Rendimiento de Empleados
# Esta consulta ayuda a identificar a los empleados con mejor desempeño en ventas, lo cual es útil para programas de incentivos, por ejemplo.
consulta_ranking_empleados_sql = """
WITH VentasNetasPorEmpleado AS (
    SELECT
        e.EmployeeID AS ID_Empleado,
        e.FirstName AS Nombre_Empleado,
        e.LastName AS Apellido_Empleado,
        SUM(s.Quantity) AS Cantidad_Total_Vendida,
        SUM(s.Quantity * p.Price * (1 - IFNULL(s.Discount, 0))) AS Valor_Total_Ventas_Netas
    FROM Sales s
    JOIN Products p ON s.ProductID = p.ProductID
    JOIN Employees e ON s.SalesPersonID = e.EmployeeID
    GROUP BY
        e.EmployeeID, e.FirstName, e.LastName
)
SELECT
    Nombre_Empleado,
    Apellido_Empleado,
    Cantidad_Total_Vendida,
    FORMAT(Valor_Total_Ventas_Netas, 2, 'es_AR') AS Valor_Total_Ventas_Netas_Formato, -- Formato moneda
    RANK() OVER (ORDER BY Valor_Total_Ventas_Netas DESC) AS Ranking_Por_Valor_Venta,
    RANK() OVER (ORDER BY Cantidad_Total_Vendida DESC) AS Ranking_Por_Cantidad_Vendida
FROM VentasNetasPorEmpleado
ORDER BY
    Ranking_Por_Valor_Venta ASC;
"""
print("\\nConsulta Avanzada: Ranking de Rendimiento de Empleados")
df_ranking_empleados = conector_bd1.ejecutar_consulta(consulta_ranking_empleados_sql)

if df_ranking_empleados is not None and not df_ranking_empleados.empty:
    print("¡Ranking de empleados obtenido exitosamente!")
    display(df_ranking_empleados)
elif df_ranking_empleados is not None:
    print("La consulta de ranking de empleados se ejecutó pero no arrojó resultados.")
else:
    print("No se pudo obtener el ranking de empleados.")

\nConsulta Avanzada: Ranking de Rendimiento de Empleados
INFO     | sistema_ventas.ConexionBD | Consulta exitosa: 23 filas en 0.049s
¡Ranking de empleados obtenido exitosamente!


,Nombre_Empleado,Apellido_Empleado,Cantidad_Total_Vendida,Valor_Total_Ventas_Netas_Formato,Ranking_Por_Valor_Venta,Ranking_Por_Cantidad_Vendida
0,Warren,Bartlett,18.0,"905,53",1,1
1,Christine,Palmer,15.0,"823,61",2,3
2,Shelby,Riddle,15.0,"807,59",3,3
3,Seth,Franco,15.0,"781,56",4,3
4,Lindsay,Chen,14.0,"747,17",5,6
5,Pablo,Cline,18.0,"689,22",6,1
6,Nicole,Fuller,9.0,"608,00",7,11
7,Chadwick,Walton,13.0,"557,14",8,7
8,Julie,Dyer,9.0,"549,62",9,11
9,Wendi,Buckley,11.0,"536,26",10,8


In [ ]:
# Consulta Avanzada 2: Segmentación de Clientes FM
# Esta segmentación ayuda a entender diferentes grupos de clientes (ej. los más leales y valiosos, los que compran esporádicamente, etc.) para dirigir estrategias de marketing o fidelización.
consulta_segmentacion_fm_sql = """
WITH ResumenComprasCliente AS (
    SELECT
        c.CustomerID AS ID_Cliente,
        c.FirstName AS Nombre_Cliente,
        c.LastName AS Apellido_Cliente,
        COUNT(DISTINCT s.SalesID) AS Frecuencia_Compras,
        SUM(s.Quantity * p.Price * (1 - IFNULL(s.Discount, 0))) AS Valor_Total_Compras
    FROM Sales s
    JOIN Products p ON s.ProductID = p.ProductID
    JOIN Customers c ON s.CustomerID = c.CustomerID
    GROUP BY
        c.CustomerID, c.FirstName, c.LastName
),
SegmentacionFM_Clientes AS (
    SELECT
        ID_Cliente,
        Nombre_Cliente,
        Apellido_Cliente,
        Frecuencia_Compras,
        Valor_Total_Compras,
        NTILE(4) OVER (ORDER BY Frecuencia_Compras DESC) AS Cuartil_Frecuencia,
        NTILE(4) OVER (ORDER BY Valor_Total_Compras DESC) AS Cuartil_Valor
    FROM ResumenComprasCliente
    WHERE Valor_Total_Compras > 0
)
SELECT
    ID_Cliente,
    Nombre_Cliente,
    Apellido_Cliente,
    Frecuencia_Compras,
    FORMAT(Valor_Total_Compras, 2, 'es_AR') AS Valor_Total_Compras_Formato,
    Cuartil_Frecuencia,
    Cuartil_Valor,
    CONCAT('F', Cuartil_Frecuencia, '-V', Cuartil_Valor) AS Segmento_FM
FROM SegmentacionFM_Clientes
ORDER BY
    Cuartil_Valor DESC, Cuartil_Frecuencia DESC, Valor_Total_Compras DESC
LIMIT 50; -- Para visualización en el notebook
"""
print("\\nConsulta Avanzada: Segmentación de Clientes por Frecuencia y Valor (FM)")
df_segmentacion_fm = conector_bd1.ejecutar_consulta(consulta_segmentacion_fm_sql) 

if df_segmentacion_fm is not None and not df_segmentacion_fm.empty:
    print("¡Segmentación FM de clientes obtenida exitosamente!")
    display(df_segmentacion_fm) 
elif df_segmentacion_fm is not None:
    print("La consulta de segmentación FM se ejecutó pero no arrojó resultados.")
else:
    print("No se pudo obtener la segmentación FM de clientes.")

\nConsulta Avanzada: Segmentación de Clientes por Frecuencia y Valor (FM)
INFO     | sistema_ventas.ConexionBD | Consulta exitosa: 50 filas en 0.202s
¡Segmentación FM de clientes obtenida exitosamente!


,ID_Cliente,Nombre_Cliente,Apellido_Cliente,Frecuencia_Compras,Valor_Total_Compras_Formato,Cuartil_Frecuencia,Cuartil_Valor,Segmento_FM
0,376,Abraham,Michael,1,"34,51",4,4,F4-V4
1,292,Robyn,Hawkins,1,"34,51",4,4,F4-V4
2,326,Dante,Chaney,1,"32,61",4,4,F4-V4
3,334,Reginald,Duke,1,"31,35",4,4,F4-V4
4,382,Amie,Carpenter,1,"30,89",4,4,F4-V4
5,378,Joanna,Bradley,1,"28,56",4,4,F4-V4
6,349,Gena,Wolf,1,"22,09",4,4,F4-V4
7,420,Austin,Butler,1,"18,38",4,4,F4-V4
8,418,Joanna,Shah,1,"16,05",4,4,F4-V4
9,448,Amanda,Gates,1,"9,97",4,4,F4-V4


INFO     | sistema_ventas.GestorCacheConsultas | INICIANDO: limpieza de cache expirado
INFO     | sistema_ventas.GestorCache | Limpieza de cache expirado completada: 3 en memoria, 3 en disco.
INFO     | sistema_ventas.GestorCacheConsultas | COMPLETADO: limpieza de cache expirado (duración: 0.038s)


In [11]:
# Creación de Vista: VistaVentasGlobalesDetalladas
# Una vista, al ser una tabla virtual, permite simplificar consultas complejas, mejorar la seguridad, y reutilizar lógicas de negocio sin duplicar código SQL.
sql_crear_vista_ventas_detalladas = """
CREATE OR REPLACE VIEW VistaVentasGlobalesDetalladas AS
SELECT
    s.SalesID AS ID_Venta,
    s.SalesDate AS Hora_Minutos_Segundos_Venta,
    prod.ProductID AS ID_Producto,
    prod.ProductName AS Nombre_Producto,
    cat.CategoryID AS ID_Categoria,
    cat.CategoryName AS Nombre_Categoria,
    cust.CustomerID AS ID_Cliente,
    CONCAT(cust.FirstName, ' ', IFNULL(CONCAT(cust.MiddleInitial, '. '), ''), cust.LastName) AS Nombre_Completo_Cliente,
    cit.CityName AS Ciudad_Cliente,
    co.CountryName AS Pais_Cliente,
    emp.EmployeeID AS ID_Empleado_Vendedor,
    CONCAT(emp.FirstName, ' ', IFNULL(CONCAT(emp.MiddleInitial, '. '), ''), emp.LastName) AS Nombre_Completo_Empleado_Vendedor,
    s.Quantity AS Cantidad_Vendida,
    prod.Price AS Precio_Unitario_Producto_Lista,
    s.Discount AS Descuento_Aplicado_Porcentaje,
    (s.Quantity * prod.Price * (1 - IFNULL(s.Discount, 0))) AS Venta_Neta_Calculada,
    s.TotalPrice AS Precio_Total_Registrado_En_Venta,
    s.TransactionNumber AS Numero_Transaccion
FROM Sales s
JOIN Products prod ON s.ProductID = prod.ProductID
JOIN Categories cat ON prod.CategoryID = cat.CategoryID
JOIN Customers cust ON s.CustomerID = cust.CustomerID
JOIN Cities cit ON cust.CityID = cit.CityID
JOIN Countries co ON cit.CountryID = co.CountryID
LEFT JOIN Employees emp ON s.SalesPersonID = emp.EmployeeID;
"""
print("\\nCreando/Reemplazando la vista 'VistaVentasGlobalesDetalladas'...")
try:
    motor = conector_bd1.obtener_motor()
    if motor:
        with motor.connect() as conexion:
            # Para SQLAlchemy 2.x, usar text() explícitamente
            from sqlalchemy import text
            conexion.execute(text(sql_crear_vista_ventas_detalladas))
            conexion.commit() # Asegurarse que el cambio se aplique
        print("Vista 'VistaVentasGlobalesDetalladas' creada/reemplazada exitosamente.")
    else:
        print("No se pudo obtener el motor de la BD.")
except Exception as e:
    print(f"Error al crear la vista: {e}")
    print("Verifica en tu cliente de BD si la vista fue creada.")

# Demostración de uso de la vista
print("\\nConsultando los primeros 5 registros de 'VistaVentasGlobalesDetalladas':")
sql_consulta_vista = "SELECT * FROM VistaVentasGlobalesDetalladas LIMIT 5;"
df_vista_resultado = conector_bd1.ejecutar_consulta(sql_consulta_vista) 

if df_vista_resultado is not None:
    display(df_vista_resultado) 
else:
    print("No se pudieron obtener datos de la vista 'VistaVentasGlobalesDetalladas'.")

\nCreando/Reemplazando la vista 'VistaVentasGlobalesDetalladas'...
Vista 'VistaVentasGlobalesDetalladas' creada/reemplazada exitosamente.
\nConsultando los primeros 5 registros de 'VistaVentasGlobalesDetalladas':
INFO     | sistema_ventas.GestorCacheConsultas | INICIANDO: búsqueda en cache de consulta
INFO     | sistema_ventas.GestorCacheConsultas | COMPLETADO: búsqueda en cache de consulta (duración: 0.002s)
INFO     | sistema_ventas.ConexionBD | HIT DE CACHE para consulta: SELECT * FROM VistaVentasGlobalesDetalladas LIMIT 5;... (0.0070s)
INFO     | sistema_ventas.ConexionBD | Consulta exitosa: 5 filas en 0.014s


,ID_Venta,Hora_Minutos_Segundos_Venta,ID_Producto,Nombre_Producto,ID_Categoria,Nombre_Categoria,ID_Cliente,Nombre_Completo_Cliente,Ciudad_Cliente,Pais_Cliente,ID_Empleado_Vendedor,Nombre_Completo_Empleado_Vendedor,Cantidad_Vendida,Precio_Unitario_Producto_Lista,Descuento_Aplicado_Porcentaje,Venta_Neta_Calculada,Precio_Total_Registrado_En_Venta,Numero_Transaccion
0,360312,0 days 00:11:43,8,Halibut - Steaks,5,Beverages,94,Paul C. Dodson,Tacoma,United States,20,Shelby P. Riddle,1,89.8573,0.0,89.8573,20.0,E1OSTE5W4DSSHM6ANJBI
1,3497041,0 days 00:45:34,144,Placemat - Scallop; White,5,Beverages,158,Demetrius G. Ramirez,Las Vegas,United States,18,Warren C. Bartlett,1,95.7865,0.0,95.7865,18.0,RFZ0FNKO7DWH2DL71BF1
2,5779347,0 days 00:34:51,144,Placemat - Scallop; White,5,Beverages,161,Sandra A. Elliott,Birmingham,United States,10,Jean P. Vang,1,95.7865,0.0,95.7865,10.0,T6J8347646PHAYLK8OYB
3,6106900,0 days 00:30:15,200,Garlic - Peeled,5,Beverages,200,Rene I. Mercado,Spokane,United States,17,Seth D. Franco,1,11.1295,0.0,11.1295,17.0,26ZF9PP6PA2OHPG9P3E1
4,4738381,0 days 00:24:25,214,French Pastry - Mini Chocolate,5,Beverages,91,Marci B. Hampton,Kansas,United States,5,Desiree L. Stuart,1,61.2121,0.0,61.2121,5.0,JLXH1JQWBTBNS9H0W3BS
